In [17]:
#You need to install all requirement modules before running this file
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from pdf2image import convert_from_bytes
from IPython.display import display, Image
import pytesseract
!sudo apt install tesseract-ocr

#Using classes in parser_generic.py
%run parser_generic.py
from parser_generic import PrescriptionParser
from parser_generic import PatientDetailsParser

#create a function to preprocessing the data
# (1)convert color image to gray image
# (2) zoom size bigger 1.5x
# (3) use adative threshold to make image clearer
def preprocess_image(img):
  gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY) #convert color image to gray
  resized = cv2.resize(gray, None,fx = 1.5, fy=1.5, interpolation=cv2.INTER_LINEAR ) #zoom the image a little bigger
  processed_image = cv2.adaptiveThreshold(
    resized, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    61, 11
)
  return processed_image

def extract(file, file_format):
  # Step 1: Extracting text from pdf file
  # convert pdf file to image
  pages = convert_from_bytes(open(file, 'rb').read(), size=2200)
  document_text = ''
  #we can have multiple pages
  if len(pages)>0:
    page = pages[0]
    processed_image = preprocess_image(page)
    #extract text from image
    lang = 'eng'
    config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(processed_image, lang = lang, config = config)
    document_text += '\n'+text

  #step 2: extracting fields from text
  if file_format == 'prescription':
    extracted_data = PrescriptionParser(document_text).parse()
  elif file_format == "patient_details":
    extracted_data = PatientDetailsParser(document_text).parse()
  else:
    raise Exception("Invalid document format : {file_format}")
  return extracted_data

if __name__== '__main__':
  data = extract('pd_2.pdf','patient_details')
  print(data)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
{'patient_name': 'Jerry Lucas \n|', 'birthday': 'May 2 1998', 'phone': '(279) 920-8204', 'hepatitis_b_vaccination': 'Yes', 'medical_probs': '- N/A |'}
